In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import multiprocessing
from torch import autograd
from fastai.conv_learner import *
from fasterai.images import *
from fasterai.dataset import *
from fasterai.visualize import *
from fasterai.callbacks import *
from fasterai.loss import *
from fasterai.modules import *
from fasterai.wgan import *
from fasterai.generators import *
from fastai.torch_imports import *
from pathlib import Path
from itertools import repeat
import tensorboardX
torch.cuda.set_device(1)
plt.style.use('dark_background')
torch.backends.cudnn.benchmark=True


In [3]:
DATA_PATH = Path('data/imagenet/ILSVRC/Data/CLS-LOC')
TRAIN_SOURCE_PATH = DATA_PATH/'train'
proj_id = 'bw2color'
TENSORBOARD_PATH = Path('data/tensorboard/EDSRWithFeatLossModel_z3')


dpath = DATA_PATH/(proj_id + '_1000_D.h5')
gpath = DATA_PATH/(proj_id + '_1000_G.h5')
keep_pct=0.1
bs=16
sz=224
wd=1e-7
lr=1e-3
lrs = np.array([lr/1000,lr/100,lr])

torch.backends.cudnn.benchmark=True

## Training

In [4]:
md = get_matched_image_model_data(image_size=sz, batch_size=bs, root_data_path=DATA_PATH, train_root_path=TRAIN_SOURCE_PATH, proj_id=proj_id, keep_pct=keep_pct)

In [5]:
netG = EDSRImageModifier(nf=32).cuda()
netGVis = ModelVisualizationHook(TENSORBOARD_PATH, netG, 'netG')
edsr = LearnerGenModuleWrapper(netG, 'edsr')
#load_model(netG, gpath)
learn = ConvLearner(md, edsr)
imgGenVis = ImageGenVisualizationCallback(TENSORBOARD_PATH, netG, md, 'learner')
learn.metrics = []
learn.opt_fn=optim.Adam
#learn.crit = F.mse_loss #(turns sepia/blurry)
learn.crit = FeatureLoss(multiplier=1e5)

In [6]:
#learn.freeze_to(1)

In [7]:
#learn.lr_find(1e-4, 1e1, wds=wd, linear=False)

In [8]:
#learn.sched.plot(n_skip=0, n_skip_end=0)

In [9]:
learn.fit(lrs, 1, cycle_len=1, wds=wd, use_clr_beta=(20,10,0.95,0.85), callbacks=[imgGenVis])

epoch      trn_loss   val_loss   
    0      86.438973  45.556526 



[45.556526463031766]